# How to remotely connect to Dice and GPU cluster?

Before starting, let we have tree types of server as follows. 
    1. Local sever (your labtop)
    2. Dice server (+ shared server)
    3. GPU server
    
### 1. From local to Dice

You can approach to Dice machine on you local server using this command.

Following this command, you should type your dice password once. 
And then, you should log in to shared server on Dice server with next command. After that, you can activate your mlp virtual environment. If you successfully connected to shared server, you can check that the name of server changed. (e.g. ashbury:~$ )

### 2. Using Jupyter Notebook on your Local server

Since jupyter notebook is run via firefox, you should use a specific command line to open jupyter notebook. 

When you type the command above, you will be able to know local host number through NotebookApp.

*Then come back to your local server and open another terminal.* In order to open jupyter notbook on your local firefox, you can use a command below.

You might be confused about the long command...
My example is as follows.



[Any_localhost_number_you_want] = 8880

[Dice_server_localhost_number] = 8888 (This is the number informed at last step.)

[name_of_server] = ashbury


Finally, you can open jupyter notebook on your local firefox via address "localhost:[Any_localhost_number_you_want]". (Type this line at address bar.)

At first, you might need to set a password for jupyter notbook because of security problem.

### 3. From Dice to GPU server

*Let's comback to dice server terminal.*
You can approach to GPU server using one of both command lines. 

Now you're on gpu server(=gpu cluster) and need to set gpu environment once following commands below.

During installation, you should type "yes" for first question and press ENTER for next question. 

Then you should follow commands below to finish setup.

If you want to use both servers, mlp1 and mlp2, you should set gpu environment on every two soervers. From next time, you can activate your environment by two commands.

Now you successfully set your gpu environment. You can run your own code using gpu by script file below.
(You should make a script file like script.sh containing scripts below.)
Please change student id, output and error file path and running command at last.

You can give a job use a sbach command line, and will get job number which is necessary to check wheter your job is running. You can see printing results from output file and error results (if error happended during running) from error file. 

Finally you can copy your result from gpu server to your dice server. 